In [1]:
import sib_ldsc_cython as ld
import sys
sys.path.append("..")
import sib_ldsc_z as ldbase

In [2]:
import numpy as np
import glob
import pandas as pd
from scipy.optimize import minimize
import scipy.optimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats

In [3]:
from importlib import reload
reload(ld)

<module 'sib_ldsc_cython' from '/disk/homedirs/nber/harij/gitrepos/SNIPar/ldsc_reg/cython/sib_ldsc_cython.cpython-37m-x86_64-linux-gnu.so'>

In [4]:
V = np.array([[0.5, 0.25], [0.25, 0.5]])
S = np.array([[0.4, 0.2], [0.2, 0.4]])

In [5]:
ldbase.standardize_mat(V, S, 100)

(array([[1.25 , 0.625],
        [0.625, 1.25 ]]),
 array([[1. , 0.5],
        [0.5, 1. ]]))

In [6]:
%time ld._log_ll(ldbase.Vmat2V(V, 100), np.array([0.1, 0.1]), S, 0.5, 100)

CPU times: user 147 ms, sys: 2 ms, total: 149 ms
Wall time: 147 ms


-2.18364641006772

In [7]:
%time ldbase._log_ll(ldbase.Vmat2V(V, 100), np.array([0.1, 0.1]), S, 0.5, 100)

CPU times: user 3.36 s, sys: 51 ms, total: 3.42 s
Wall time: 3.41 s


-2.18364641006772

In [8]:
ld._grad_ll_v(ldbase.Vmat2V(V, 100), np.array([0.1, 0.1]), S, 0.5, 100)

array([-0.00383037, -0.00383037,  0.2574622 ])

In [9]:
ldbase._grad_ll_v(ldbase.Vmat2V(V, 100), np.array([0.1, 0.1]), S, 0.5, 100)

array([-0.00383037, -0.00383037,  0.2574622 ])

In [10]:
Smat = np.array([[[0.4, 0.2], [0.2, 0.4]]] * 100)

model = ldbase.sibreg(Smat)
model.simdata(V, 100)
z = model.z

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for LD Scores given. Generating a vector of ones for l
No value for effective number of loci is given. Using total number of loci instead


In [15]:
%time ld.neg_logll_grad(ld.Vmat2V(V, 100), z, Smat, np.ones(100), np.ones(100), 100)

CPU times: user 32.9 ms, sys: 2.07 ms, total: 35 ms
Wall time: 33 ms


(366.67598850939777, array([ 0.02937249, -0.2091432 , 16.00795093]))

In [16]:
%time ldbase.neg_logll_grad(ldbase.Vmat2V(V, 100), z, Smat, np.ones(100), np.ones(100), 100)

CPU times: user 3 ms, sys: 0 ns, total: 3 ms
Wall time: 2.97 ms


(366.67598850939777, array([ 0.02937249, -0.2091432 , 16.00795093]))